In [1]:
import ray, random, os 
from ray.train.xgboost import XGBoostTrainer
from ray.air.config import ScalingConfig
import pandas as pd
ray.init(_temp_dir='/Volumes/SSD980/ray')
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL
from ray.train.rl import RLTrainer
from ray.rllib.policy.policy import Policy, PolicySpec
from ray.tune.registry import register_env
from ray.tune import TuneConfig
from ray.tune.logger import pretty_print
from ray.tune.search.bayesopt import BayesOptSearch
from ray.air.config import RunConfig, CheckpointConfig
from ray.tune.schedulers import HyperBandScheduler
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.search.hyperopt import HyperOptSearch

2023-09-08 17:32:38,168	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-09-08 17:32:44,227	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [4]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def create_policy_spec(worker_id):
    print(f"Creating policy for {worker_id} with obs space {env.observation_space[worker_id]} and action space {env.action_space[worker_id]}")
    return PolicySpec(
        observation_space=env.observation_space[worker_id],
        action_space=env.action_space[worker_id],
        config={}
    )


manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "manager_policy": manager_policy_spec,
}
for worker_id in env.workers:
    policies[worker_id] = create_policy_spec(worker_id)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return agent_id
    else:
        return "manager_policy"

search_space = {
     "lr": tune.uniform(1e-4, 1e-1),
    "gamma": tune.uniform(0.9, 0.99),
    "lambda": tune.uniform(0.9, 1.0),
    "entropy_coeff": tune.uniform(1e-4, 1e-1),
    "vf_loss_coeff": tune.uniform(0.1, 1.0),
}

config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    search_alg=BayesOptSearch(),
    # scheduler=HyperBandScheduler(),
    num_samples=10,
    max_concurrent_trials=3
    )

run_config = RunConfig(
    name="Test",
    storage_path="/Volumes/SSD980/ray/results/test",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=10),

)

tuner = tune.Tuner(
    trainable="A2C",
    param_space={
        "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "lr": tune.uniform(1e-4, 1e-1),
        "gamma": tune.uniform(0.9, 0.99),
        "lambda": tune.uniform(0.9, 1.0),
        "entropy_coeff": tune.uniform(1e-4, 1e-1),
        "vf_loss_coeff": tune.uniform(0.1, 1.0),
        "num_workers": 3,
    },
    run_config=run_config,
    # If tune_config is custom, you might still use it, but make sure it's correctly implemented
    tune_config=config
)
results = tuner.fit()
print(results.get_best_result(metric="episode_reward_mean", mode="max").config)

2023-09-08 17:46:30,181	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-09-08 17:46:30,186	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2023-09-08 17:46:30,259	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


Creating policy for ABT.US with obs space Dict('current_cash': Box(-inf, inf, (1,), float32), 'current_price': Box(-inf, inf, (1,), float32), 'current_stock_exposure': Box(-inf, inf, (1,), float32), 'day': Box(-inf, inf, (1,), float32), 'pnl': Box(-inf, inf, (1,), float32), 'return_per_volatility': Box(-inf, inf, (1,), float32), 'shares_held': Box(-inf, inf, (1,), float32), 'tech_indicators': Box(-inf, inf, (14,), float32), 'total_costs': Box(-inf, inf, (1,), float32), 'total_trades': Box(-inf, inf, (1,), float32)) and action space Dict('amount': Box(0.0, 1.0, (1,), float32), 'type': Discrete(3))
Creating policy for AMGN.US with obs space Dict('current_cash': Box(-inf, inf, (1,), float32), 'current_price': Box(-inf, inf, (1,), float32), 'current_stock_exposure': Box(-inf, inf, (1,), float32), 'day': Box(-inf, inf, (1,), float32), 'pnl': Box(-inf, inf, (1,), float32), 'return_per_volatility': Box(-inf, inf, (1,), float32), 'shares_held': Box(-inf, inf, (1,), float32), 'tech_indicators':

/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
2023-09-08 17:46:30,346	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-09-08 17:46:30,376	INFO t

== Status ==
Current time: 2023-09-08 17:46:31 (running for 00:00:00.94)
Using FIFO scheduling algorithm.
Logical resource usage: 0/8 CPUs, 0/0 GPUs
Result logdir: /Volumes/SSD980/ray/results/test/Test
Number of trials: 1/10 (1 PENDING)
+------------------+----------+-------+-----------------+----------+----------+----------+-----------------+
| Trial name       | status   | loc   |   entropy_coeff |    gamma |   lambda |       lr |   vf_loss_coeff |
|------------------+----------+-------+-----------------+----------+----------+----------+-----------------|
| A2C_hrl_d82fc844 | PENDING  |       |       0.0375166 | 0.985564 | 0.973199 | 0.059906 |        0.240417 |
+------------------+----------+-------+-----------------+----------+----------+----------+-----------------+




(pid=27142) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=27142) 2023-09-08 17:46:40,400	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=27142) 2023-09-08 17:46:40,400	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=27142) 2023-09-08 17:46:40,400	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.


...

== Status ==
Current time: 2023-09-08 17:49:37 (running for 00:03:07.10)
Using FIFO scheduling algorithm.
Logical resource usage: 4.0/8 CPUs, 0/0 GPUs
Result logdir: /Volumes/SSD980/ray/results/test/Test
Number of trials: 10/10 (9 ERROR, 1 PENDING)
+------------------+----------+-----------------+-----------------+----------+----------+-----------+-----------------+
| Trial name       | status   | loc             |   entropy_coeff |    gamma |   lambda |        lr |   vf_loss_coeff |
|------------------+----------+-----------------+-----------------+----------+----------+-----------+-----------------|
| A2C_hrl_57e8e16b | PENDING  |                 |      0.066286   | 0.928054 | 0.952007 | 0.0547164 |        0.266369 |
| A2C_hrl_d82fc844 | ERROR    | 127.0.0.1:27142 |      0.0375166  | 0.985564 | 0.973199 | 0.059906  |        0.240417 |
| A2C_hrl_c02bf1da | ERROR    | 127.0.0.1:27211 |      0.0156839  | 0.905228 | 0.986618 | 0.0601514 |        0.737265 |
| A2C_hrl_422dc560 | ERROR    |

(RolloutWorker pid=27602) 2023-09-08 17:49:38,796	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=27602) 2023-09-08 17:49:38,796	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=27602) 2023-09-08 17:49:38,802	WARNING deprecation.py:50 -- DeprecationWarning: `StochasticSampling` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=27602) 2023-09-08 17:49:38,802	WARNING deprecation.py:50 -- DeprecationWarning: `Exploration` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=27602) 2023-09-08 17:49:38,802	WARNING deprecation.py:50 -- DeprecationWarning: `Random` has been deprecated. This will raise an error in the 

== Status ==
Current time: 2023-09-08 17:49:44 (running for 00:03:14.57)
Using FIFO scheduling algorithm.
Logical resource usage: 4.0/8 CPUs, 0/0 GPUs
Result logdir: /Volumes/SSD980/ray/results/test/Test
Number of trials: 10/10 (10 ERROR)
+------------------+----------+-----------------+-----------------+----------+----------+-----------+-----------------+
| Trial name       | status   | loc             |   entropy_coeff |    gamma |   lambda |        lr |   vf_loss_coeff |
|------------------+----------+-----------------+-----------------+----------+----------+-----------+-----------------|
| A2C_hrl_d82fc844 | ERROR    | 127.0.0.1:27142 |      0.0375166  | 0.985564 | 0.973199 | 0.059906  |        0.240417 |
| A2C_hrl_c02bf1da | ERROR    | 127.0.0.1:27211 |      0.0156839  | 0.905228 | 0.986618 | 0.0601514 |        0.737265 |
| A2C_hrl_422dc560 | ERROR    | 127.0.0.1:27259 |      0.00215639 | 0.987292 | 0.983244 | 0.0213127 |        0.263642 |
| A2C_hrl_9ffc9c5e | ERROR    | 127.0.0.1

2023-09-08 17:49:47,060	WARNING experiment_analysis.py:916 -- Failed to read the results for 10 trials:
- /Volumes/SSD980/ray/results/test/Test/A2C_hrl_d82fc844_1_entropy_coeff=0.0375,env=hrl,gamma=0.9856,lambda=0.9732,lr=0.0599,ABT_US=ref_ph_0c4b22f9,AMGN_US=ref_ph_fd750e6_2023-09-08_17-46-30
- /Volumes/SSD980/ray/results/test/Test/A2C_hrl_c02bf1da_2_entropy_coeff=0.0157,env=hrl,gamma=0.9052,lambda=0.9866,lr=0.0602,ABT_US=ref_ph_0c4b22f9,AMGN_US=ref_ph_fd750e6_2023-09-08_17-46-53
- /Volumes/SSD980/ray/results/test/Test/A2C_hrl_422dc560_3_entropy_coeff=0.0022,env=hrl,gamma=0.9873,lambda=0.9832,lr=0.0213,ABT_US=ref_ph_0c4b22f9,AMGN_US=ref_ph_fd750e6_2023-09-08_17-47-12
- /Volumes/SSD980/ray/results/test/Test/A2C_hrl_9ffc9c5e_4_entropy_coeff=0.0184,env=hrl,gamma=0.9274,lambda=0.9525,lr=0.0433,ABT_US=ref_ph_0c4b22f9,AMGN_US=ref_ph_fd750e6_2023-09-08_17-47-29
- /Volumes/SSD980/ray/results/test/Test/A2C_hrl_1f95c59e_5_entropy_coeff=0.0612,env=hrl,gamma=0.9126,lambda=0.9292,lr=0.0367,ABT_US=

RuntimeError: No best trial found for the given metric: episode_reward_mean. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.

In [2]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
     "lr": tune.uniform(1e-4, 1e-1),
    "gamma": tune.uniform(0.9, 0.99),
    "lambda": tune.uniform(0.9, 1.0),
    "entropy_coeff": tune.uniform(1e-4, 1e-1),
    "vf_loss_coeff": tune.uniform(0.1, 1.0),
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=10,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Volumes/SSD980/ray/results",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=300),

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
          "lr": tune.uniform(1e-5,1e-4),
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

analysis = tune.run("A2C", metric="episode_reward_mean", mode="max",config=param_space)

2023-09-08 16:26:28,293	WARNING bayesopt_search.py:201 -- You passed a `space` parameter to <class 'ray.tune.search.bayesopt.bayesopt_search.BayesOptSearch'> that contained unresolved search space definitions. <class 'ray.tune.search.bayesopt.bayesopt_search.BayesOptSearch'> should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `param_space` argument to `tune.Tuner()` instead.
2023-09-08 16:26:28,294	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The LogUniform sampler will be dropped.
2023-09-08 16:26:28,294	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The LogUniform sampler will be dropped.
2023-09-08 16:26:28,300	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https

2023-09-08 16:26:28,638	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property num_cpus_per_trial not supported.
(pid=15321) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=15321) 2023-09-08 16:26:36,724	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=15321) 2023-09-08 16:26:36,725	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=15321) 2023-09-08 16:26:36,725	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property num_cpus_per_trial not supported.
(pid=15330) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=15330) 2023-09-

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,timers
A2C_hrl_b2ed3_00000,1960448,"{'ObsPreprocessorConnector_ms': 0.07513273846019398, 'StateBufferConnector_ms': 0.008488243276422674, 'ViewRequirementAgentConnector_ms': 0.3082307902249423}","{'num_env_steps_sampled': 140032, 'num_env_steps_trained': 140032, 'num_agent_steps_sampled': 1960448, 'num_agent_steps_trained': 1960448}",{},6304,{},724775,397985,22836.6,0,"{'learner': {'manager_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 40.0, 'cur_lr': 7.135231685121378e-05, 'entropy_coeff': 0.01, 'policy_entropy': 8983.3740234375, 'policy_loss': 13770.2509765625, 'vf_loss': 47.276458740234375}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 32, 'num_grad_updates_lifetime': 4376, 'diff_num_grad_updates_vs_sampler_policy': 4375}, 'worker_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 40.0, 'cur_lr': 7.135231685121378e-05, 'entropy_coeff': 0.01, 'policy_entropy': -79.89375305175781, 'policy_loss': -8641.462890625, 'vf_loss': 18568436.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 416, 'num_grad_updates_lifetime': 4376, 'diff_num_grad_updates_vs_sampler_policy': 4375}}, 'num_env_steps_sampled': 140032, 'num_env_steps_trained': 140032, 'num_agent_steps_sampled': 1960448, 'num_agent_steps_trained': 1960448}",1960448,1960448,140032,576,56.963,140032,576,56.963,0,1,0,0,576,"{'cpu_util_percent': 47.05714285714286, 'ram_util_percent': 84.4}","{'manager_policy': 2316.146240569651, 'worker_policy': 234259.65842725144}","{'manager_policy': 1531.562260591698, 'worker_policy': 30496.449637810987}","{'manager_policy': 418.42606826680094, 'worker_policy': -4245.862063456094}","{'mean_raw_obs_processing_ms': 1.9023988075349936, 'mean_inference_ms': 4.685624718198401, 'mean_action_processing_ms': 1.7296003001411258, 'mean_env_wait_ms': 10.432182731786636, 'mean_env_render_ms': 0.0}","{'training_iteration_time_ms': 613.34, 'sample_time_ms': 596.347, 'learn_time_ms': 15.738, 'learn_throughput': 2033.322, 'synch_weights_time_ms': 1.04}"


(RolloutWorker pid=15330) =========HRL is done=============
(RolloutWorker pid=15330) HRL is done
(RolloutWorker pid=15330) day: 6303, episode: 2
(RolloutWorker pid=15330) Total Cash Transfers: 48
(RolloutWorker pid=15330) total_portfolio_trades: 49654.0
(RolloutWorker pid=15330) Beginn_Portfolio_Value: 2000000
(RolloutWorker pid=15330) End_Portfolio_Value: 15056719.0
(RolloutWorker pid=15330) Annual Return: 12.40 %
(RolloutWorker pid=15330) Worker ID: ABT.US Current Stock Exposure: 756047
(RolloutWorker pid=15330) Worker ID: AMGN.US Current Stock Exposure: 0
(RolloutWorker pid=15330) Worker ID: BDX.US Current Stock Exposure: 677528
(RolloutWorker pid=15330) Worker ID: BMY.US Current Stock Exposure: 94354
(RolloutWorker pid=15330) Worker ID: HUM.US Current Stock Exposure: 632814
(RolloutWorker pid=15330) Worker ID: JNJ.US Current Stock Exposure: 124153
(RolloutWorker pid=15330) Worker ID: LLY.US Current Stock Exposure: 374805
(RolloutWorker pid=15330) Worker ID: MDT.US Current Stock Ex

(raylet) Spilled 2083 MiB, 78 objects, write throughput 203 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


...

(RolloutWorker pid=15330) =========HRL is done=============
(RolloutWorker pid=15330) HRL is done
(RolloutWorker pid=15330) day: 6303, episode: 10
(RolloutWorker pid=15330) Total Cash Transfers: 25
(RolloutWorker pid=15330) total_portfolio_trades: 36786.0
(RolloutWorker pid=15330) Beginn_Portfolio_Value: 2000000
(RolloutWorker pid=15330) End_Portfolio_Value: 60118648.0
(RolloutWorker pid=15330) Annual Return: 21.78 %
(RolloutWorker pid=15330) Worker ID: ABT.US Current Stock Exposure: 2578313
(RolloutWorker pid=15330) Worker ID: AMGN.US Current Stock Exposure: 21588166
(RolloutWorker pid=15330) Worker ID: BDX.US Current Stock Exposure: 3549859
(RolloutWorker pid=15330) Worker ID: BMY.US Current Stock Exposure: 1035378
(RolloutWorker pid=15330) Worker ID: HUM.US Current Stock Exposure: 844491
(RolloutWorker pid=15330) Worker ID: JNJ.US Current Stock Exposure: 2832456
(RolloutWorker pid=15330) Worker ID: LLY.US Current Stock Exposure: 1586373
(RolloutWorker pid=15330) Worker ID: MDT.US Cu

(raylet) Spilled 4116 MiB, 154 objects, write throughput 262 MiB/s.


(RolloutWorker pid=15330) =========HRL is done=============
(RolloutWorker pid=15330) HRL is done
(RolloutWorker pid=15330) day: 6303, episode: 15
(RolloutWorker pid=15330) Total Cash Transfers: 59
(RolloutWorker pid=15330) total_portfolio_trades: 7084.0
(RolloutWorker pid=15330) Beginn_Portfolio_Value: 2000000
(RolloutWorker pid=15330) End_Portfolio_Value: 40424148.0
(RolloutWorker pid=15330) Annual Return: 19.01 %
(RolloutWorker pid=15330) Worker ID: ABT.US Current Stock Exposure: 476536
(RolloutWorker pid=15330) Worker ID: AMGN.US Current Stock Exposure: 9026491
(RolloutWorker pid=15330) Worker ID: BDX.US Current Stock Exposure: 2148455
(RolloutWorker pid=15330) Worker ID: BMY.US Current Stock Exposure: 663190
(RolloutWorker pid=15330) Worker ID: HUM.US Current Stock Exposure: 739318
(RolloutWorker pid=15330) Worker ID: JNJ.US Current Stock Exposure: 2652716
(RolloutWorker pid=15330) Worker ID: LLY.US Current Stock Exposure: 522317
(RolloutWorker pid=15330) Worker ID: MDT.US Current

2023-09-08 17:09:12,451	WARNING util.py:315 -- The `callbacks.on_trial_result` operation took 0.582 s, which may be a performance bottleneck.
2023-09-08 17:09:12,454	WARNING util.py:315 -- The `process_trial_result` operation took 0.584 s, which may be a performance bottleneck.
2023-09-08 17:09:12,454	WARNING util.py:315 -- Processing trial results took 0.585 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2023-09-08 17:09:12,454	WARNING util.py:315 -- The `process_trial_result` operation took 0.585 s, which may be a performance bottleneck.


(RolloutWorker pid=15330) =========HRL is done=============
(RolloutWorker pid=15330) HRL is done
(RolloutWorker pid=15330) day: 6303, episode: 21
(RolloutWorker pid=15330) Total Cash Transfers: 42
(RolloutWorker pid=15330) total_portfolio_trades: 7501.0
(RolloutWorker pid=15330) Beginn_Portfolio_Value: 2000000
(RolloutWorker pid=15330) End_Portfolio_Value: 51478380.0
(RolloutWorker pid=15330) Annual Return: 20.69 %
(RolloutWorker pid=15330) Worker ID: ABT.US Current Stock Exposure: 1827636
(RolloutWorker pid=15330) Worker ID: AMGN.US Current Stock Exposure: 6718482
(RolloutWorker pid=15330) Worker ID: BDX.US Current Stock Exposure: 2862534
(RolloutWorker pid=15330) Worker ID: BMY.US Current Stock Exposure: 924099
(RolloutWorker pid=15330) Worker ID: HUM.US Current Stock Exposure: 1013567
(RolloutWorker pid=15330) Worker ID: JNJ.US Current Stock Exposure: 2915210
(RolloutWorker pid=15330) Worker ID: LLY.US Current Stock Exposure: 1363565
(RolloutWorker pid=15330) Worker ID: MDT.US Curr

In [3]:
from ray.tune import CLIReporter
def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
env = HRL()

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune_config,
    param_space=param_space,

    run_config=run_config,
)
results = tuner.fit()


2023-09-07 22:38:47,616	WARNING bayesopt_search.py:201 -- You passed a `space` parameter to <class 'ray.tune.search.bayesopt.bayesopt_search.BayesOptSearch'> that contained unresolved search space definitions. <class 'ray.tune.search.bayesopt.bayesopt_search.BayesOptSearch'> should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `param_space` argument to `tune.Tuner()` instead.
2023-09-07 22:38:47,617	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The LogUniform sampler will be dropped.
2023-09-07 22:38:47,651	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-09-07 22:38:47,654	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2023-09-07 22:38:47,692	WARNING algorithm_config.

== Status ==
Current time: 2023-09-07 22:38:48 (running for 00:00:00.86)
Using HyperBand: num_stopped=0 total_brackets=1
Round #0:
  Bracket(Max Size (n)=5, Milestone (r)=81, completed=0.0%): {PENDING: 1} 
Logical resource usage: 0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuner_trial/MyExperiment
Number of trials: 1/100 (1 PENDING)
+------------------+----------+-------+----------+-----------+
| Trial name       | status   | loc   |    gamma |        lr |
|------------------+----------+-------+----------+-----------|
| A2C_hrl_b9c02a0d | PENDING  |       | 0.933709 | 0.0950764 |
+------------------+----------+-------+----------+-----------+


== Status ==
Current time: 2023-09-07 22:38:53 (running for 00:00:05.88)
Using HyperBand: num_stopped=0 total_brackets=1
Round #0:
  Bracket(Max Size (n)=5, Milestone (r)=81, completed=0.0%): {PENDING: 1} 
Logical resource usage: 2.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuner_trial/MyExperiment
N

(pid=80275) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=80275) 2023-09-07 22:38:54,989	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=80275) 2023-09-07 22:38:54,989	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=80275) 2023-09-07 22:38:54,990	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property num_cpus_per_trial not supported.


== Status ==
Current time: 2023-09-07 22:38:58 (running for 00:00:10.98)
Using HyperBand: num_stopped=0 total_brackets=1
Round #0:
  Bracket(Max Size (n)=5, Milestone (r)=81, completed=0.0%): {PENDING: 1} 
Logical resource usage: 2.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuner_trial/MyExperiment
Number of trials: 1/100 (1 PENDING)
+------------------+----------+-------+----------+-----------+
| Trial name       | status   | loc   |    gamma |        lr |
|------------------+----------+-------+----------+-----------|
| A2C_hrl_b9c02a0d | PENDING  |       | 0.933709 | 0.0950764 |
+------------------+----------+-------+----------+-----------+




(pid=80285) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=80285) 2023-09-07 22:39:01,008	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=80285) 2023-09-07 22:39:01,008	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=80285) 2023-09-07 22:39:01,027	WARNING deprecation.py:50 -- DeprecationWarning: `StochasticSampling` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=80285) 2023-09-07 22:39:01,027	WARNING deprecation.py:50 -- DeprecationWarning: `Exploration` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=80285) 2023-09-07 22:39:01,

...

2023-09-07 22:39:37,678	ERROR tune_controller.py:911 -- Trial task failed for trial A2C_hrl_74a821b2
Traceback (most recent call last):
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/worker.py", line 2524, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(ValueError): ray::A2C.train() (pid=80328, ip=127.0.0.1, actor_id=e25146695b16b5104b4517ef01000000, repr=A2C)
  File "/Users/floriankockler/anaconda3

== Status ==
Current time: 2023-09-07 22:39:39 (running for 00:00:51.45)
Using HyperBand: num_stopped=0 total_brackets=1
Round #0:
  Bracket(Max Size (n)=5, Milestone (r)=81, completed=0.0%): {ERROR: 3, PENDING: 1} 
Logical resource usage: 2.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuner_trial/MyExperiment
Number of trials: 4/100 (3 ERROR, 1 PENDING)
+------------------+----------+-----------------+----------+-----------+
| Trial name       | status   | loc             |    gamma |        lr |
|------------------+----------+-----------------+----------+-----------|
| A2C_hrl_d077352f | PENDING  |                 | 0.905228 | 0.086631  |
| A2C_hrl_b9c02a0d | ERROR    | 127.0.0.1:80275 | 0.933709 | 0.0950764 |
| A2C_hrl_869de91c | ERROR    | 127.0.0.1:80304 | 0.965879 | 0.059906  |
| A2C_hrl_74a821b2 | ERROR    | 127.0.0.1:80328 | 0.914042 | 0.0156839 |
+------------------+----------+-----------------+----------+-----------+
Number of errored trials: 3
+--------

(pid=80378) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=80378) 2023-09-07 22:39:43,877	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=80378) 2023-09-07 22:39:43,877	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=80378) 2023-09-07 22:39:43,878	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property num_cpus_per_trial not supported.


== Status ==
Current time: 2023-09-07 22:39:44 (running for 00:00:56.54)
Using HyperBand: num_stopped=0 total_brackets=1
Round #0:
  Bracket(Max Size (n)=5, Milestone (r)=81, completed=0.0%): {ERROR: 3, PENDING: 1} 
Logical resource usage: 2.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tuner_trial/MyExperiment
Number of trials: 4/100 (3 ERROR, 1 PENDING)
+------------------+----------+-----------------+----------+-----------+
| Trial name       | status   | loc             |    gamma |        lr |
|------------------+----------+-----------------+----------+-----------|
| A2C_hrl_d077352f | PENDING  |                 | 0.905228 | 0.086631  |
| A2C_hrl_b9c02a0d | ERROR    | 127.0.0.1:80275 | 0.933709 | 0.0950764 |
| A2C_hrl_869de91c | ERROR    | 127.0.0.1:80304 | 0.965879 | 0.059906  |
| A2C_hrl_74a821b2 | ERROR    | 127.0.0.1:80328 | 0.914042 | 0.0156839 |
+------------------+----------+-----------------+----------+-----------+
Number of errored trials: 3
+--------

(pid=80387) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=80387) 2023-09-07 22:39:50,974	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=80387) 2023-09-07 22:39:50,974	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=80387) 2023-09-07 22:39:50,979	WARNING deprecation.py:50 -- DeprecationWarning: `StochasticSampling` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=80387) 2023-09-07 22:39:50,979	WARNING deprecation.py:50 -- DeprecationWarning: `Exploration` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=80387) 2023-09-07 22:39:50,

In [ ]:
import random
import os
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hrl", env_creator)

manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }
env = HRL(hrl_config)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 


first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}


def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    #"rollout_fragment_length": [50, 100, 200, 400],
    "train_batch_size": lambda: random.randint(200, 1500),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),

}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env_config": hrl_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 1,  # 1 for training + 4 for sampling
        "num_cpus_per_trial": 3,
        # "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        # "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/autobatch", progress_reporter=reporter),
)
results = tuner.fit()